## 4.5 Overdetermined Systems

We use the implementation of the QR factorization based on Householder transformations. This also covers the rectangular case.

In [ ]:
import numpy as np
from scripts.lu import backward, forward2, cholesky

#### Example 4.27 (Best approximation of an overdetermined system using QR factorisations)

We consider the overdetermined system $Ax = b$ with
$$
A = \begin{pmatrix}
  1 & -\frac{1}{4} & \frac{1}{16} \\
  1 & \frac{1}{2} & \frac{1}{4} \\
  1 & 2 & 4 \\
  1 & \frac{5}{2} &\frac{25}{4} 
\end{pmatrix},\quad b=\begin{pmatrix} 0 \\1 \\ 0 \\ 1
\end{pmatrix}.
$$

In [ ]:
A = np.array([[1, -1 / 4, 1 / 16],
              [1, 1 / 2,  1 / 4],
              [1, 2,      4],
              [1, 5 / 2,  25 / 4]], dtype=np.half)
b = np.array([0, 1, 0, 1], dtype=np.half)
A2 = A.copy()

First, we set up the system of normal equations $A^T A = A^T b$

In [ ]:
ATA = A.transpose() @ A
ATb = np.inner(A.transpose(), b)

And solve this using the Cholesky factorization

In [ ]:
L = cholesky(ATA)
y = forward2(L, ATb)
x_L = backward(L.transpose(), y)
print(x_L)

This results in the residual norm

In [ ]:
print(f'||Ax - b|| = {np.linalg.norm(np.inner(A, x_L) - b):.4e}')

Now, we solve the system using the QR factorization using Householder transformations using
$$
Rx = Q^T b.
$$
This requires the computation of the modified right-hand side $\tilde{b}=Q^Tb$, which we compute using the following algorithm

In [ ]:
from scripts.qr import qr_householder

def b_tilde(V, b):
    bt = b.copy()
    m = A.shape[1]
    for i in range(m):
        bt[i:] -= 2 * np.inner(V[i:, i], bt[i:]) * V[i:, i]
    return np.array(bt[:m])

Using `half` precision, we then get the solution

In [ ]:
A = np.array([[1, -1 / 4, 1 / 16],
              [1, 1 / 2,  1 / 4],
              [1, 2,      4],
              [1, 5 / 2,  25 / 4]], dtype=np.half)
b = np.array([0, 1, 0, 1], dtype=np.half)

V = qr_householder(A)
bt = b_tilde(V, b)
x_h = backward(A[:V.shape[1],:], bt)
print(f'x = {x_h}')

In [ ]:
print(f'||Ax - b|| = {np.linalg.norm(np.inner(A2, x_h) - b):.4e}')

This yields the following difference to the solution of the normal equation

In [ ]:
print(np.linalg.norm(x_h - x_L))

We can proceed similarly using the QR factorization using Givens rotations

In [ ]:
from scripts.qr import qr_givens

A = np.array([[1, -1 / 4, 1 / 16],
              [1, 1 / 2,  1 / 4],
              [1, 2,      4],
              [1, 5 / 2,  25 / 4]], dtype=np.half)
b = np.array([0, 1, 0, 1], dtype=np.half)

QT = qr_givens(A)
bt = np.dot(QT, b)[:A.shape[1]]
x_g = backward(A[:A.shape[1],:], bt)
print(f'x = {x_g}')

This results in the residual norm

In [ ]:
print(f'||Ax - b|| = {np.linalg.norm(np.inner(A2, x_g) - b):.4e}')

and the difference to the solution computed using the QR factorization with Householder transformations is

In [ ]:
print(np.linalg.norm(x_h - x_g))